In [ ]:
from astropy.time import Time, TimeDelta
import astropy.units as u
import holoviews as hv
from holoviews import opts
hv.extension('bokeh', logo=False)
import numpy as np
import pandas as pd

from lsst_efd_client import EfdClient

In [ ]:
start_date = Time("2025-01-17T03:33:00", scale="utc")
time_span = 600
end_date= start_date + TimeDelta(val=time_span * u.s)
tel = "AT"
camera = f"{tel}Camera"
hs = f"{tel}HeaderService"
camera_topics = ["logevent_startIntegration", "logevent_endReadout", "logevent_endOfImageTelemetry"]
hs_topics = ["logevent_largeFileObjectAvailable"]
efd_name = "base_efd"

In [ ]:
client = EfdClient(efd_name)

In [ ]:
data = []
for topic in camera_topics:
    df = await client.select_time_series(f"lsst.sal.{camera}.{topic}",
                                         "imageName",
                                         start_date,
                                         end_date)
    data.append(df)
for topic in hs_topics:
    df = await client.select_time_series(f"lsst.sal.{hs}.{topic}",
                                         "url",
                                         start_date,
                                         end_date)
    image_name = []
    for v in df["url"]:
        filename = v.split("/")[-1]
        img = "_".join(filename.split(".")[0].split("_")[2:])
        image_name.append(img)
    df = df.drop(columns="url").assign(imageName=image_name)
    data.append(df)

In [ ]:
d1 = hv.Table(data[0].reset_index())
si = hv.Points(d1, label="Start Integration").opts(marker="+")

d2 = hv.Table(data[1].reset_index())
er = hv.Points(d2, label="End Readout").opts(marker="x")

d3 = hv.Table(data[2].reset_index())
eit = hv.Points(d3, label="End Image Telemetry").opts(marker="^")

d4 = hv.Table(data[3].reset_index())
lfoa = hv.Points(d4, label="Header Available").opts(marker="v")

In [ ]:
layout = si * er * eit * lfoa
layout.opts(opts.Points(size=20, width=1000, height=600, legend_position="left", legend_offset=(0, 200),
                        xlabel="Time", ylabel="Image Name"))

In [ ]:
diff = data[-1].index - data[-2].index
diff_sec = diff / np.timedelta64(1, "ns") / 1e9

In [ ]:
df_diff = pd.DataFrame(data={"imageName": data[-1]["imageName"].values, "TimeDiff": diff_sec})
dtp = hv.Points(df_diff, label="Time Difference Between Camera End of Image Telemetry and HS LFOA").opts(marker="o")

In [ ]:
dtp.opts(opts.Points(size=20, width=1000, height=600, xlabel="Image Name", xrotation=90, ylabel="ΔT (sec)"))